In [ ]:
import hashlib
import random
import pyprowl

def sha3_hash(text):
    """
    Calculates the SHA-3 hash of a given text.

    Args:
    text (str): The text to hash.

    Returns:
    bytes: The SHA-3 hash value.
    """
    # Encode the text as bytes
    text_bytes = text.encode('utf-8')

    # Calculate the SHA-3 hash
    hash_value = hashlib.sha3_256(text_bytes).digest()

    return hash_value

def reduce_sha3(hash_value, charset='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'):

  # Validate and convert the hash value to bytes
  clean_hash = ''.join(c for c in hash_value if c in '0123456789abcdefABCDEF')  # Remove invalid characters
  hash_bytes = bytes.fromhex(clean_hash)

  # Use hash for some randomness while maintaining consistency
  seed = int(clean_hash[:4], 16)  # Use first 4 bytes for seeding (after cleaning)
  random.seed(seed)

  reduced_string = ""
  reduced_string += ''.join(random.sample(charset, 5))  # Randomly select 5 characters

  # Limit random XOR value and ensure it maps to a valid character
  random_value = (int(clean_hash[:2], 16) ^ random.randrange(seed)) % len(charset)
  reduced_string += charset[random_value]  # Directly select character from charset

  reduced_string += ''.join(random.choices(charset, k=len(hash_bytes) // 2))  # Randomly select half the hash length characters

  return reduced_string

def create_table():
  file = open('most_used_merged_unique.txt_v2.txt', 'r')
  rainbowTable = open('rainbowtable.txt', 'a')

  while True :
    content=file.readline().strip()
    beginning = content
    if not content:
      break
    sha3=""
    reduced = ""
    for x in range(0, 10000):
      sha3 = sha3_hash(content)
      content = reduce_sha3(sha3.hex())
    endOfCall = sha3_hash(content)
    rainbowTable.write("" + endOfCall.hex() + ":" + beginning + '\n')
  file.close()
  rainbowTable.close()

def try_find_hash(str_hash):
    rainbowTable = open('rainbowtable.txt', 'r')
    while True:
      content=rainbowTable.readline().strip()
      beginning = content
      if not content:
        return None
      row = content.split(":")
      if(row[0] == str_hash):
        return row[1]
def crack_hash(str_hash):
  original_hash = str_hash
  while True:
    result = try_find_hash(str_hash)
    if(result == None):
      str_hash = reduce_sha3(str_hash)
      str_hash = sha3_hash(str_hash)
    if(result != None):
      print('Matching chain found')
      parse_chain_to_password(original_hash, result)
      break

def parse_chain_to_password(original_hash, beginning_of_chain):
  previousValue = ""
  while True:
    previousValue = beginning_of_chain
    sha3_hash_chain = sha3_hash(beginning_of_chain)
    if(sha3_hash_chain == original_hash):
      notify_cracked(original_hash, previousValue)
      break
    else:
      beginning_of_chain = reduce_sha3(sha3_hash_chain)

def notify_cracked(hash, password):
  p = pyprowl.Prowl('API-KEY')
  try:
    p.verify_key()
    print("Prowl API key successfully verified!")
  except Exception as e:
    print("Error verifying Prowl API key: {}".format(e))
    exit()

  try:
    p.notify(event='Password cracked', description='Password for SHA3-Hash: hash is password',
        priority=0, appName='Google Colab')
    print("Notification successfully sent to Prowl!")
  except Exception as e:
    print("Error sending notification to Prowl: {}".format(e))


crack_hash((sha3_hash("soccer")).hex())


ValueError: invalid literal for int() with base 16: b'\xd4A'